In [1]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
import random
import math

import sys
sys.path.append('../')
import saedfsc

Here is a notional list of suppliers.

In [2]:
saedfsc.suppliers

,Name,Location,Rating,SetUpCost
0,Supplier1,Location1,1.178742,4303
1,Supplier2,Location2,2.930649,2174
2,Supplier3,Location1,4.821298,2699
3,Supplier4,Location4,2.837146,2724
4,Supplier5,Location2,3.659921,4715
5,Supplier6,Location2,1.560158,2056
6,Supplier7,Location3,1.165378,3938


Customer information

In [3]:
saedfsc.customers

,Name,Quantity,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,469,0.391040,"[0.91450978,0.08733943,0.51950344,0.55561313,0...","[0.34496792,0.6898727 ,0.13100207,0.50656613,0..."
1,CustomerType2,929,0.951341,"[0.66601538,0.54644132,0.24889524,0.86383297,0...","[0.46336316,0.47837937,0.80161025,0.20513706,0..."
2,CustomerType3,294,0.740209,"[0.52786049,0.73410731,0.85361372,0.83415508,0...","[0.92694124,0.48651615,0.78058322,0.45806116,0..."
3,CustomerType4,834,0.608726,"[0.30815853,0.44577241,0.59348252,0.60900538,0...","[0.45334148,0.17085646,0.67141338,0.78225234,0..."
4,CustomerType5,733,0.565038,"[0.7610932 ,0.05656771,0.61822428,0.8374061 ,0...","[0.64583732,0.09631073,0.23214996,0.88206262,0..."
5,CustomerType6,35,0.254575,"[0.5931458 ,0.67793322,0.22446516,0.4948187 ,0...","[0.57868043,0.07927113,0.05214455,0.78084477,0..."
6,CustomerType7,239,0.362928,"[0.66483611,0.90905972,0.18731687,0.66342118,0...","[0.54702712,0.05151569,0.09804964,0.04074776,0..."
7,CustomerType8,564,0.770436,"[0.76932255,0.36910923,0.89955738,0.1623181 ,0...","[0.56254937,0.13950323,0.18176402,0.34446781,0..."
8,CustomerType9,927,0.599208,"[0.51947281,0.1023688 ,0.59192487,0.60865134,0...","[0.53924253,0.90808112,0.34176956,0.47190584,0..."
9,CustomerType10,366,0.452926,"[0.09527526,0.13840712,0.67149965,0.23045829,0...","[0.09699416,0.73592342,0.92129521,0.89635862,0..."


Quantity discount information

In [4]:
saedfsc.qtyDiscountSchedule

,Unnamed: 0,Supplier,Broader Part,Specific Part: Row Index,Price Level,MinQty,Discount
0,0,Supplier1,wings,885,1,9,0.10
1,1,Supplier1,wings,885,2,18,0.26
2,2,Supplier1,wings,885,3,31,0.42
3,3,Supplier1,wings,1354,1,8,0.14
4,4,Supplier1,wings,1354,2,20,0.28
...,...,...,...,...,...,...,...
922,922,Supplier7,impactattenuator,318,2,20,0.27
923,923,Supplier7,impactattenuator,318,3,36,0.39
924,924,Supplier7,impactattenuator,787,1,5,0.12
925,925,Supplier7,impactattenuator,787,2,18,0.22


In [5]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

In [6]:
partOptions['tires']

,Unnamed: 0_x,Pressure Rating,pressure,pressure (Ordering Pressure Ratings),Unnamed: 0_y,ID,radius,mass,cost,radius (Ordering tires),Suppliers
0,0,Basic,0.758,0.758,0,T1,0.22860,3.636,300,0.22860,"[Supplier3, Supplier4, Supplier2]"
1,0,Basic,0.758,0.758,1,T2,0.22987,4.091,302,0.22987,[Supplier5]
2,0,Basic,0.758,0.758,2,T3,0.23241,4.545,306,0.23241,[Supplier4]
3,0,Basic,0.758,0.758,3,T4,0.24638,4.545,324,0.24638,"[Supplier5, Supplier4, Supplier2]"
4,0,Basic,0.758,0.758,4,T5,0.24765,5.000,326,0.24765,"[Supplier6, Supplier4, Supplier7]"
5,0,Basic,0.758,0.758,5,T8,0.26162,5.455,344,0.26162,[Supplier5]
6,0,Basic,0.758,0.758,6,T6,0.26670,5.000,352,0.26670,[Supplier1]
7,1,Low,0.834,0.834,0,T1,0.22860,3.636,300,0.22860,[Supplier3]
8,1,Low,0.834,0.834,1,T2,0.22987,4.091,302,0.22987,"[Supplier7, Supplier4]"
9,1,Low,0.834,0.834,2,T3,0.23241,4.545,306,0.23241,[Supplier3]


Chad: I put in numbers for placeholders. Please put the actual numbers in. (Note: if the qtyDiscountSchedule contains the discount values rather than prices, then we need to set the nominal prices. But another option is to have the qtyDiscountSchedule include prices instead of discounts. You can do whatever you think is easier.)

In [7]:
# Create Nominal Part Prices Dictionary (assuming all broad parts will be included)
broadPartNames = ['wings', 'tires', 'engine', 'cabin', 'brakes', 'impactattenuator', 'suspension']

# Define function to create nominalPartPrices dictionary
def make_nominal_part_prices(broad_part_names, part_options):
    nominalPartPrices = {}
    for broad_part in broad_part_names:
        broad_part_price_dict = {}
        for specific_parts in range(len(partOptions[broad_part])):
            if broad_part == 'wings':
                
                broad_part_price_dict[part_options['wings'].index[specific_parts]] = round(part_options['wings'].iloc[specific_parts]['length'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['width'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['height'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['q'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['cost_per_kilogram'], 2)
            
            elif broad_part == 'tires':
                
                broad_part_price_dict[part_options['tires'].index[specific_parts]] = round(part_options['tires'].iloc[specific_parts]['cost'] * 
                                                                                     (1 + (part_options['tires'].iloc[specific_parts]['pressure'] - 0.758) / 2), 2)
                                            
                
            elif broad_part == 'engine':
                
                broad_part_price_dict[part_options['engine'].index[specific_parts]] = round(part_options['engine'].iloc[specific_parts]['Cost'], 2)
                
            elif broad_part == 'cabin':
                
                broad_part_price_dict[part_options['cabin'].index[specific_parts]] = round(2 * 
                                                                                     part_options['cabin'].iloc[specific_parts]['thickness'] * 
                                                                                     (part_options['cabin'].iloc[specific_parts]['length'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['width'] + 
                                                                                      part_options['cabin'].iloc[specific_parts]['length'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['height'] + 
                                                                                      part_options['cabin'].iloc[specific_parts]['width'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['height']  
                                                                                     ) *
                                                                                     part_options['cabin'].iloc[specific_parts]['q'] * 
                                                                                     part_options['cabin'].iloc[specific_parts]['cost_per_kilogram'], 2)
                
            elif broad_part == 'brakes':
                
                broad_part_price_dict[part_options['brakes'].index[specific_parts]] = round(part_options['brakes'].iloc[specific_parts]['lbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['wbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['hbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['qbrk'] * 
                                                                                      1000 * 
                                                                                      25, 2)
                                            
                
            elif broad_part == 'impactattenuator':
                
                broad_part_price_dict[part_options['impactattenuator'].index[specific_parts]] = round(part_options['impactattenuator'].iloc[specific_parts]['length'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['width'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['height'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['q'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['cost_per_kilogram'], 2)
                
                
            elif broad_part == 'suspension':
                
                broad_part_price_dict[part_options['suspension'].index[specific_parts]] = 0 # Assuming suspension has fixed cost and is "tuned" (Comment from Dr. McComb's code)
            
            
        nominalPartPrices[broad_part] = broad_part_price_dict
        
    return nominalPartPrices

In [8]:
# Example use of function make_nominal_part_prices

nominalPartPrices = make_nominal_part_prices(broadPartNames, partOptions)

# To access the cost of a specific part, index by the broad part (wings, engine, etc)
# and then by the row index of the specific part

# Could be useful if row index was reset once sample was taken for partOptions

for subsystem in partOptions.keys():
    print(subsystem)
    print(nominalPartPrices[subsystem])
    print()

wings
{909: 11.52, 396: 1060.32, 885: 2.88, 1255: 1395.9, 1354: 40.34, 2591: 12587.93, 1642: 335.28, 28: 1.05, 2532: 2113.94, 1858: 15087.6, 680: 7.8, 748: 144.02, 335: 170.77, 2106: 84239.1, 1381: 116.43, 299: 57.4, 1669: 731.52, 540: 131.44, 1226: 644.65, 1143: 42.98}

tires
{0: 300.0, 1: 302.0, 2: 306.0, 3: 324.0, 4: 326.0, 5: 344.0, 6: 352.0, 7: 311.4, 8: 313.48, 9: 317.63, 10: 336.31, 11: 338.39, 12: 357.07, 13: 365.38, 14: 321.9, 15: 324.05, 16: 328.34, 17: 347.65, 18: 349.8, 19: 369.11, 20: 377.7, 21: 331.8, 22: 334.01, 23: 338.44, 24: 358.34, 25: 360.56, 26: 380.46, 27: 389.31, 28: 340.8, 29: 343.07, 30: 347.62, 31: 368.06, 32: 370.34, 33: 390.78, 34: 399.87}

engine
{0: 650, 1: 880, 2: 870, 3: 880, 4: 870, 5: 915, 6: 900, 7: 915, 8: 915, 9: 1020, 10: 1020, 11: 1020, 12: 915, 13: 1350, 14: 1380, 15: 1350, 16: 1350, 17: 1380, 18: 1150, 19: 1650, 20: 1650}

cabin
{995: 499.61, 464: 378.92, 107: 85.7, 1677: 10754.04, 1783: 24958.22, 553: 395.14, 1326: 507.11, 177: 53.64, 1786: 291

Parameters

In [9]:
productPrice = 20000
maxProductPrice = 25000
pricePerf = (maxProductPrice - productPrice) / maxProductPrice

Data structures

In [10]:
suppliers = saedfsc.suppliers['Name'].to_list()
supplierSetUpCost = dict(zip(saedfsc.suppliers['Name'], saedfsc.suppliers['SetUpCost']))
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
unique_triples_df = saedfsc.qtyDiscountSchedule[['Supplier', 'Broader Part', 'Specific Part: Row Index']].drop_duplicates()
suppliersPartTriples = list(unique_triples_df.itertuples(index=False, name=None))

Chad: fill this in

In [11]:
objectives = ['mass', 'center of gravity', 'drag', 'downforce', 'acceleration', 
              'crash force', 'attenuator volume', 'cornering velocity', 
              'braking distance', 'suspension acceleration', 'pitch moment'] # fill this in with the objectives you want to optimize

Pricing schedule

In [12]:
priceLevels = {}
minQtys = {}
prices = {}
for s,bp,p in suppliersPartTriples:
    filtered_df = saedfsc.qtyDiscountSchedule[(saedfsc.qtyDiscountSchedule['Supplier'] == s) & (saedfsc.qtyDiscountSchedule['Specific Part: Row Index'] == p) & (saedfsc.qtyDiscountSchedule['Broader Part'] == bp)]
    priceLevels[s,bp,p] = filtered_df['Price Level'].tolist()
    minQtys[s,bp,p] = filtered_df.set_index('Price Level')['MinQty'].to_dict()

discounts = saedfsc.qtyDiscountSchedule.set_index(['Supplier', 'Broader Part', 'Specific Part: Row Index','Price Level'])['Discount'].to_dict()

maxQty = {}
for s,bp,p in suppliersPartTriples:
    maxQty[s,bp,p] = 1000

suppliersPartsAndLevels = []
for s,bp,p in suppliersPartTriples:
    for l in priceLevels[s,bp,p]:
        suppliersPartsAndLevels.append((s,bp,p,l))

Assumptions for car performance aspect of model

In [13]:
random.seed(10) # Since uniform distribution used in center of gravity constraints

v_car = 26.8 #m/s
omega_e = 3600 #rpm
rho_air = 1.225 #kg/m^3
r_track = 9 #m
P_brk = 1 * (10 ** 7) #Pa
g = 9.81 #m/s^2
y_parameter = 0.05 #m
y_dot_parameter = 0.025 #m/s
c_brk = 0.37 # Coefficient of Brake Friction
l_f = 0.5 # Pitch Moment Parameter
C_lat = 1.6 # Cornering Velocity parameter in Dr. McComb code

# Car performance objectives minimum values
objectives_min = [95.4413,
                  0.1159,
                  4.8283,
                  0.006299,
                  0.0,
                  1636425.0,
                  0.004049,
                  0.01908,
                  5.9088,
                  9.81,
                  0.03386]

# Car performance objectives maximum values
objectives_max = [5593.26,
                   0.9996,
                   633.95,
                   3211.16,
                   4.35,
                   672530217,
                   0.1579,
                   15.0652,
                   553.92,
                   18.79,
                   9850.87]

# Weights of each car performance term in objective function (Scenario 1)
performance_weights = [0.14, 0.01, 0.20, 0.30, 0.10, 0.01, 0.01, 0.10, 0.10, 0.02, 0.01] 


Create model container.

In [14]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-26


Create variables

In [15]:
x = {} # part selection variables
for subsystem in partOptions.keys():
    if subsystem == 'wings':
        x['rear wing'] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[rear wing]")
        x['front wing'] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[front wing]")
        x['side wing'] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[side wing]")
    elif subsystem == 'tires':
        x['front tire'] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[front tire]")
        x['rear tire'] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[rear tire]")
    else:
        x[subsystem] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[" + subsystem + "]")
y = m.addVars(suppliers, vtype=GRB.BINARY, name="y") # supplier selection variables
z = m.addVars(objectives, vtype = GRB.CONTINUOUS, name="z") # objective variables (normalized to [0,1] for minimization)
v = m.addVars(suppliersPartsAndLevels, name="v") # the purchase quantity, given that the quantity of part p purchased from supplier s is in price level l
vBin = m.addVars(suppliersPartsAndLevels, vtype = GRB.BINARY, name="vBin") # if the quantity of part p purchased from supplier s is in price level l
w = m.addVars(customers, ub = cQty, name="w") # variables to hold customer demand
u_e_mTotal = m.addVars(partOptions['engine'].index.values, vtype = GRB.CONTINUOUS, name = "u_e_mTotal") # x_e * mTotal Equivalency Variable
u_rootMass = m.addVar(1, vtype = GRB.CONTINUOUS, name = 'u_rootMass') # square root of total mass equivalence variable
u_rt_brkDistance = m.addVars(partOptions['tires'].index.values, vtype = GRB.CONTINUOUS, name = 'u_rt_downforce') # Rear Tire and Downforce Equivalency Variable
u_rt_pressure_inverse = m.addVars(partOptions['tires'].index.values, vtype = GRB.CONTINUOUS, name = 'u_rt_pressure_inverse') # Inverse of rear tire pressure for C calculation
u_rw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_rw_c') # rear wing and cabin equivalence variable
u_fw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_fw_c') # front wing and cabin equivalence variable
u_sw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_sw_c') # side wing and cabin equivalence variable

Normalized car performance objectives

In [16]:
normalized_car_performance_objectives = ((performance_weights[0] * ((z['mass'] - objectives_min[0]) / (objectives_max[0] - objectives_min[0]))) + 
                              (performance_weights[1] * ((z['center of gravity'] - objectives_min[1]) / (objectives_max[1] - objectives_min[1]))) + 
                              (performance_weights[2] * ((z['drag'] - objectives_min[2]) / (objectives_max[2] - objectives_min[2]))) + 
                              (performance_weights[3] * ((-z['downforce'] + objectives_max[3]) / (-objectives_min[3] + objectives_max[3]))) +
                              (performance_weights[4] * ((-z['acceleration'] + objectives_max[4]) / (-objectives_min[4] + objectives_max[4]))) + 
                              (performance_weights[5] * ((z['crash force'] - objectives_min[5]) / (objectives_max[5] - objectives_min[5]))) +
                              (performance_weights[6] * ((z['attenuator volume'] - objectives_min[6]) / (objectives_max[6] - objectives_min[6]))) + 
                              (performance_weights[7] * ((-z['cornering velocity'] + objectives_max[7]) / (-objectives_min[7] + objectives_max[7]))) + 
                              (performance_weights[8] * ((z['braking distance'] - objectives_min[8]) / (objectives_max[8] - objectives_min[8]))) + 
                              (performance_weights[9] * ((z['suspension acceleration'] - objectives_min[9]) / (objectives_max[9] - objectives_min[9]))) + 
                              (performance_weights[10] * ((z['pitch moment'] - objectives_min[10]) / (objectives_max[10] - objectives_min[10]))))
                   

Set objective

In [17]:
m.setObjective(qsum(productPrice*cQty[c]*w[c] for c in customers) - qsum(nominalPartPrices[bp][p]*discounts[s,bp,p,l]*v[s,bp,p,l] for s,bp,p,l in suppliersPartsAndLevels) - y.prod(supplierSetUpCost), GRB.MAXIMIZE)

Quantity discount constraints

In [18]:
numLevels = {(s,bp,p) : len(priceLevels[s,bp,p]) for s,bp,p in suppliersPartTriples}

suppliersPartsAndLevelsNotLast = [(s,bp,p,l) for s,bp,p in suppliersPartTriples for l in range(1,numLevels[s,bp,p])]

m.addConstrs((minQtys[s,bp,p][l]*vBin[s,bp,p,l] <= v[s,bp,p,l] for s,bp,p,l in suppliersPartsAndLevels), 
                "PriceBreaks-LB")
m.addConstrs((v[s,bp,p,l] <= minQtys[s,bp,p][l+1]*vBin[s,bp,p,l] for s,bp,p,l in suppliersPartsAndLevelsNotLast), 
                "PriceBreaks-UB")
m.addConstrs((v[s,bp,p,numLevels[s,bp,p]] <= maxQty[s,bp,p]*vBin[s,bp,p,numLevels[s,bp,p]] for s,bp,p in suppliersPartTriples), 
                "PriceBreaks-UB-last");

Chad: add code here to set the equations for the $z$ variables (objectives); you should add your other constraints involving the objectives.

In [19]:
# Mass Constraints

rw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['front wing'][part] for part in partOptions['wings'].index.values)

sw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['rear wing'][part] for part in partOptions['wings'].index.values)

ft_mass = gp.quicksum(partOptions['tires'].loc[part]['mass'] * 
                      x['front tire'][part] for part in partOptions['tires'].index.values)

rt_mass = gp.quicksum(partOptions['tires'].loc[part]['mass'] * 
                      x['rear tire'][part] for part in partOptions['tires'].index.values)

c_mass = 2 * gp.quicksum(partOptions['cabin'].loc[part]['thickness'] * 
                         (partOptions['cabin'].loc[part]['length'] * 
                          partOptions['cabin'].loc[part]['width'] + 
                          partOptions['cabin'].loc[part]['length'] * 
                          partOptions['cabin'].loc[part]['height'] + 
                          partOptions['cabin'].loc[part]['width'] * 
                          partOptions['cabin'].loc[part]['height']) * 
                         partOptions['cabin'].loc[part]['q'] * 
                         x['cabin'][part] for part in partOptions['cabin'].index.values)

e_mass = gp.quicksum(partOptions['engine'].loc[part]['Mass'] * 
                     x['engine'][part] for part in partOptions['engine'].index.values)

ia_mass = gp.quicksum(partOptions['impactattenuator'].loc[part]['length'] * 
                      partOptions['impactattenuator'].loc[part]['width'] * 
                      partOptions['impactattenuator'].loc[part]['height'] * 
                      partOptions['impactattenuator'].loc[part]['q'] * 
                      x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

brk_mass = gp.quicksum(partOptions['brakes'].loc[part]['lbrk'] * 
                       partOptions['brakes'].loc[part]['wbrk'] * 
                       partOptions['brakes'].loc[part]['hbrk'] * 
                       partOptions['brakes'].loc[part]['qbrk'] * 
                       1000 * 
                       x['brakes'][part] for part in partOptions['brakes'].index.values)
                      
rsp_mass = gp.quicksum(partOptions['suspension'].loc[part]['mrsp'] * 
                       x['suspension'][part] for part in partOptions['suspension'].index.values)

fsp_mass = gp.quicksum(partOptions['suspension'].loc[part]['mfsp'] * 
                       x['suspension'][part] for part in partOptions['suspension'].index.values)
                      
total_mass = (rw_mass + fw_mass + (2 * sw_mass) + (2 * ft_mass) + (2 * rt_mass) +
             c_mass + e_mass + ia_mass + (4 * brk_mass) + (2 * rsp_mass) + (2 * fsp_mass))
    
m.addConstr(z['mass'] == total_mass)
m.update()

In [20]:
# Center of Gravity Constraints

rw_position_y = gp.quicksum(np.random.uniform(0.5 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              1.2 - (partOptions['wings'].loc[part]['height'] / 2)) *
                                              x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              0.25 - (partOptions['wings'].loc[part]['height'] / 2)) *
                                              x['front wing'][part] for part in partOptions['wings'].index.values)

sw_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              0.25 - (partOptions['wings'].loc[part]['height'] / 2)) * 
                                              x['side wing'][part] for part in partOptions['wings'].index.values)

rt_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                            x['rear tire'][part] for part in partOptions['tires'].index.values)

ft_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                            x['front tire'][part] for part in partOptions['tires'].index.values)

c_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['cabin'].loc[part]['height'] / 2), 
                                             1.2 - (partOptions['cabin'].loc[part]['height'] / 2)) * 
                                             x['cabin'][part] for part in partOptions['cabin'].index.values)

e_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['engine'].loc[part]['Height'] / 2), 
                                             0.50 - (partOptions['engine'].loc[part]['Height'] / 2)) * 
                                             x['engine'][part] for part in partOptions['engine'].index.values)

ia_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['impactattenuator'].loc[part]['height'] / 2), 
                                              1.20 - (partOptions['impactattenuator'].loc[part]['height'] / 2)) * 
                                              x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

brk_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                             x['front tire'][part] for part in partOptions['tires'].index.values)

rsp_position_y = gp.quicksum(np.random.uniform(partOptions['tires'].loc[part]['radius'], 
                                               2 * partOptions['tires'].loc[part]['radius']) * 
                                               x['rear tire'][part] for part in partOptions['tires'].index.values)

fsp_position_y = gp.quicksum(np.random.uniform(partOptions['tires'].loc[part]['radius'], 
                                               2 * partOptions['tires'].loc[part]['radius']) * 
                                               x['front tire'][part] for part in partOptions['tires'].index.values)

m.addConstr(z['center of gravity'] * z['mass'] == (rw_mass * rw_position_y) + 
                                                  (fw_mass * fw_position_y) + 
                                                  (2 * sw_mass * sw_position_y) + 
                                                  (2 * ft_mass * ft_position_y) + 
                                                  (2 * rt_mass * rt_position_y) + 
                                                  (c_mass * c_position_y) + 
                                                  (e_mass * e_position_y) + 
                                                  (ia_mass * ia_position_y) + 
                                                  (4 * brk_mass * brk_position_y) + 
                                                  (2 * rsp_mass * rsp_position_y) + 
                                                  (2 * fsp_mass * fsp_position_y))
m.update()

In [21]:
# Drag Constraints

rw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['front wing'][part] for part in partOptions['wings'].index.values)

sw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['side wing'][part] for part in partOptions['wings'].index.values)

c_drag = gp.quicksum(0.02 * 
                     rho_air * 
                     (v_car ** 2) * 
                     partOptions['cabin'].loc[part]['width'] * 
                     partOptions['cabin'].loc[part]['height'] * 
                     x['cabin'][part] for part in partOptions['cabin'].index.values)

total_drag = rw_drag + fw_drag + (2 * sw_drag) + c_drag

m.addConstr(z['drag'] == total_drag)
m.update()

In [22]:
# Downforce Constraints

rw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['front wing'][part] for part in partOptions['wings'].index.values)

sw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['side wing'][part] for part in partOptions['wings'].index.values)

total_downforce = rw_downforce + fw_downforce + (2 * sw_downforce)

m.addConstr(z['downforce'] == total_downforce)
m.update()

In [23]:
# Acceleration Constraint

C = (0.005 + (sum(u_rt_pressure_inverse[part] for part in partOptions['tires'].index.values)) *
            (0.01 + (0.0095 * (((v_car * 3.6) / (100)) ** 2))))

F_wheels = gp.quicksum((partOptions['engine'].loc[part]['Torque'] * 
                       (3600 * 2 * math.pi / 60) / 
                       (v_car)) * 
                       x['engine'][part] for part in partOptions['engine'].index.values)

total_resistance = (gp.quicksum(total_drag * x['engine'][part] for part in partOptions['engine'].index.values) + 
                   gp.quicksum(C * g * u_e_mTotal[part] for part in partOptions['engine'].index.values))
    
m.addConstr(z['acceleration'] * z['mass'] == F_wheels - total_resistance)
m.addConstrs(u_e_mTotal[part] == z['drag'] * x['engine'][part] for part in partOptions['engine'].index.values)
m.addConstrs(u_rt_pressure_inverse[part] * partOptions['tires'].loc[part]['pressure'] * x['rear tire'][part] == x['rear tire'][part] for part in partOptions['tires'].index.values)
m.update()

In [24]:
# Crash Force Constraints

m.addConstr(z['crash force'] == gp.quicksum((math.sqrt((v_car ** 2) * 
                                                       partOptions['impactattenuator'].loc[part]['width'] * 
                                                       partOptions['impactattenuator'].loc[part]['height'] * 
                                                       partOptions['impactattenuator'].loc[part]['E'] / 
                                                       (2 * partOptions['impactattenuator'].loc[part]['length']))) * 
                                            u_rootMass * 
                                            x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values))
                                                  
m.addConstr(u_rootMass * u_rootMass == z['mass'])
m.update()

In [25]:
# Impact Attenuator Volume Constraints

attenuator_volume = gp.quicksum(partOptions['impactattenuator'].loc[part]['length'] * 
                                partOptions['impactattenuator'].loc[part]['width'] * 
                                partOptions['impactattenuator'].loc[part]['height'] * 
                                x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

m.addConstr(z['attenuator volume'] == attenuator_volume)
m.update()

In [26]:
# Cornering Velocity Constraints

v_cor_forces = gp.quicksum((z['downforce'] + 
                            (z['mass'] * g) - 
                            (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                  y_dot_parameter * partOptions['suspension'].loc[part]['crsp'])) - 
                            (2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                  y_dot_parameter * partOptions['suspension'].loc[part]['cfsp']))) * 
                            x['suspension'][part] for part in partOptions['suspension'].index.values)
            
m.addConstr(z['cornering velocity'] * z['mass'] == v_cor_forces * C_lat * r_track)
m.update()

In [27]:
# Braking Distance

F_y = ((z['mass'] * g) + 
       (z['downforce']) - 
       (gp.quicksum(2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                         y_dot_parameter * partOptions['suspension'].loc[part]['crsp']) * 
                    x['suspension'][part] for part in partOptions['suspension'].index.values)) - 
       (gp.quicksum(2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                         y_dot_parameter * partOptions['suspension'].loc[part]['cfsp']) * 
                    x['suspension'][part] for part in partOptions['suspension'].index.values)))

lhs_remainder = (gp.quicksum(u_rt_brkDistance[part] * 
                             partOptions['tires'].loc[part]['radius'] * 
                             (0.005 + 
                              (1 / partOptions['tires'].loc[part]['pressure']) * 
                              (0.01 + 0.0095 * (((v_car * 3.6) / (100)) ** 2))) for part in partOptions['tires'].index.values))

lhs = F_y * lhs_remainder

m.addConstr(lhs == ((gp.quicksum((v_car ** 2) * 
                                 (partOptions['tires'].loc[part]['radius'] * 
                                  x['rear tire'][part]) * 
                                (z['mass']) for part in partOptions['tires'].index.values)) - 
                    (gp.quicksum(z['braking distance'] * 
                                 8 * 
                                 c_brk * 
                                 P_brk * 
                                 partOptions['brakes'].loc[part]['hbrk'] * 
                                 partOptions['brakes'].loc[part]['wbrk'] * 
                                 partOptions['brakes'].loc[part]['rbrk'] * 
                                 x['brakes'][part] for part in partOptions['brakes'].index.values))))

m.addConstrs(u_rt_brkDistance[part] == z['braking distance'] * x['rear tire'][part] for part in partOptions['tires'].index.values)
m.update()

In [28]:
# Suspension Acceleration Constraints

m.addConstr(z['suspension acceleration'] * z['mass'] == (gp.quicksum(((-2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                                                            y_dot_parameter * partOptions['suspension'].loc[part]['cfsp'])) + 
                                                                     (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                                                           y_dot_parameter * partOptions['suspension'].loc[part]['crsp'])) + 
                                                                     (z['mass'] * g) + 
                                                                     (z['downforce'])) * 
                                                                    x['suspension'][part] for part in partOptions['suspension'].index.values)))
m.update()

In [29]:
# Pitch Moment Constraints

sp_pitch_moment = gp.quicksum(((2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                    y_dot_parameter * partOptions['suspension'].loc[part]['cfsp'])) + 
                              (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                    y_parameter * partOptions['suspension'].loc[part]['crsp']))) * 
                             x['suspension'][part] for part in partOptions['suspension'].index.values)

rw_pitch_moment = gp.quicksum(rw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_rw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

fw_pitch_moment = gp.quicksum(fw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_fw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

sw_pitch_moment = gp.quicksum(sw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_fw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

pitch_moment_total = sp_pitch_moment + rw_pitch_moment - fw_pitch_moment - (2 * sw_pitch_moment)

m.addConstrs(u_rw_c[wing, cabin] == x['rear wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)
m.addConstrs(u_fw_c[wing, cabin] == x['front wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)
m.addConstrs(u_sw_c[wing, cabin] == x['side wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

m.addConstr(z['pitch moment'] == pitch_moment_total)
m.update()

In [30]:
# Part selection constraints

m.addConstr(gp.quicksum(x['rear wing'][part] for part in partOptions['wings'].index.values) == 1)
m.addConstr(gp.quicksum(x['front wing'][part] for part in partOptions['wings'].index.values) == 1)
m.addConstr(gp.quicksum(x['side wing'][part] for part in partOptions['wings'].index.values) == 1)
m.addConstr(gp.quicksum(x['rear tire'][part] for part in partOptions['tires'].index.values) == 1)
m.addConstr(gp.quicksum(x['front tire'][part] for part in partOptions['tires'].index.values) == 1)
m.addConstr(gp.quicksum(x['cabin'][part] for part in partOptions['cabin'].index.values) == 1)
m.addConstr(gp.quicksum(x['engine'][part] for part in partOptions['engine'].index.values) == 1)
m.addConstr(gp.quicksum(x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values) == 1)
m.addConstr(gp.quicksum(x['brakes'][part] for part in partOptions['brakes'].index.values) == 1)
m.addConstr(gp.quicksum(x['suspension'][part] for part in partOptions['suspension'].index.values) == 1)

# Set model non-convex parameter = 2
m.params.NonConvex = 2

Set parameter NonConvex to value 2


Hugh: add code to compute customer demand based on utility

In [31]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1868 rows, 3419 columns and 4288 nonzeros
Model fingerprint: 0xce34edda
Model has 1299 quadratic constraints
Variable types: 1055 continuous, 2364 integer (2364 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+04]
  QMatrix range    [5e-03, 1e+07]
  QLMatrix range   [5e-02, 8e+04]
  Objective range  [8e-02, 2e+07]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 1e+00]
Presolve removed 1855 rows and 1887 columns
Presolve time: 0.05s
Presolved: 83155 rows, 29194 columns, 218486 nonzeros
Presolved model has 558 SOS constraint(s)
Presolved model has 811 bilinear constraint(s)
Variable types: 1557 continuous, 27637 integer (27637 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Ro

In [32]:
# Print car performance objectives

for obj in objectives:
    print(obj, round(z[obj].X, 4))

mass 129.3778
center of gravity 0.4428
drag 10.2942
downforce 0.0
acceleration 0.0356
crash force 7229795.3732
attenuator volume 0.05
cornering velocity 45.5444
braking distance 45.7268
suspension acceleration 9.81
pitch moment 890.0
